In [ ]:
import numpy as np
import pandas as pd
import pandas_ta as ta


In [ ]:
source_df = pd.read_csv(f'./dataset.csv')
grouped_dfs = [group for _, group in source_df.groupby('crypto_name')]
crypto_names = [group['crypto_name'].unique() for group in grouped_dfs]
crypto_names2 = list(map(lambda x: x[0], crypto_names))
crypto_names2

In [ ]:
crypto_names_selection = ['Bitcoin', 'Ethereum', 'Dogecoin', 'XRP']

In [ ]:
crypto_dfs = []
for df in grouped_dfs:
    if(df['crypto_name'].iloc[0] not in crypto_names_selection):
        continue;
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    df.drop(columns=['Unnamed: 0'], inplace=True)
    crypto_dfs.append(df)
print(f"{len(crypto_dfs)} dfs, {len(crypto_names_selection)} crypto_names_selection")
# dfs[0].columns
crypto_dfs[0]

In [ ]:
sample_crypto_df = crypto_dfs[0]

Calculate the required technical indicators. For the the patterns, we need to calculate the Simple Moving Average (SMA) and Relative Strength Index (RSI) indicators.

In [ ]:
def add_sma_rsi(df, window):
    # Calculate SMA
    sma = ta.sma(df['close'], window=window)
    df[f'SMA{window}'] = sma

    # Calculate RSI
    rsi = ta.rsi(df['close'], window=window)
    df[f'RSI{window}'] = rsi

    return df

# Add SMA and RSI columns for a window
sample_crypto_df = add_sma_rsi(sample_crypto_df, window=20)


Create a function to detect the patterns

In [ ]:
def detect_descending_triangle(high, low, close, window):
    pattern = [0] * len(close)
    for i in range(window, len(close)):
        left_highs = high[i-window:i]
        right_highs = high[i-window+1:i+1]
        lows = low[i-window:i+1]
        if max(left_highs) >= max(right_highs) and min(lows[:-1]) == min(lows):
            pattern[i] = -1
        elif max(left_highs) <= max(right_highs) and max(lows[:-1]) == max(lows):
            pattern[i] = 1
    return pattern

sample_crypto_df['descending_triangle'] = detect_descending_triangle(sample_crypto_df['high'], sample_crypto_df['low'], sample_crypto_df['close'], window=20)

# Identify the occurrence of the pattern using the dataframe.loc method to find the index positions where the pattern occurs.
bullish_descending_triangle_indexes = data.loc[data['descending_triangle'] == 1].index
bearish_descending_triangle_indexes = data.loc[data['descending_triangle'] == -1].index

In [ ]:
def detect_double(lowOrHigh):
    pattern = [0] * len(lowOrHigh)
    for i in range(2, len(lowOrHigh)-2):
        if lowOrHigh[i-2] > lowOrHigh[i] == lowOrHigh[i+2] > lowOrHigh[i+1] and lowOrHigh[i+1] > lowOrHigh[i]:
            pattern[i] = 1
    return pattern

In [ ]:
sample_len = len(sample_crypto_df)
sample_len

In [ ]:
sample_crypto_df['double_bottom'] = detect_double(sample_crypto_df['low'])
bullish_double_bottom_indexes = sample_crypto_df.loc[sample_crypto_df['double_bottom'] == 1].index
print(f'bullish_double_bottom_indexes: {len(bullish_double_bottom_indexes)}')

In [ ]:
sample_crypto_df['double_top'] = detect_double(sample_crypto_df['high'])
bullish_double_top_indexes = sample_crypto_df.loc[sample_crypto_df['double_top'] == 1].index
print(f'bullish_double_top_indexes: {len(bullish_double_top_indexes)}')

In [ ]:
def detect_head_and_shoulders(high, low, close, window_size=20):
    pattern = [0] * len(close)
    for i in range(window_size, len(close) - window_size):
        left_peak = high[i-window_size:i+1].max()
        right_peak = high[i-window_size+1:i+2+window_size].max()
        head = high[i-window_size+1:i+2+window_size].min()
        left_trough = low[i-window_size:i+1].min()
        right_trough = low[i-window_size+1:i+2+window_size].min()
        if left_peak > right_peak and left_trough == right_trough == head:
            pattern[i] = 1
        elif left_peak < right_peak and left_trough == right_trough == head:
            pattern[i] = -1
    return pattern


sample_crypto_df['head_and_shoulders'] = detect_head_and_shoulders(sample_crypto_df['high'], sample_crypto_df['low'], sample_crypto_df['close'])

# Identify the occurrence of the pattern using the dataframe.loc method to find the index positions where the pattern occurs.
bullish_head_and_shoulders_indexes = sample_crypto_df.loc[sample_crypto_df['head_and_shoulders'] == 1].index
bearish_head_and_shoulders_indexes = sample_crypto_df.loc[sample_crypto_df['head_and_shoulders'] == -1].index
print(f'bullish_head_and_shoulders_indexes: {len(bullish_head_and_shoulders_indexes)}, bearish_head_and_shoulders_indexes: {len(bearish_head_and_shoulders_indexes)}')

In [ ]:
def detect_symmetrical_triangle(high, low):
    pattern = [0] * len(high)
    for i in range(2, len(high)-2):
        if high[i] == high[i-2] and low[i] == low[i-2]:
            continue
        slope1 = (high[i-1] - low[i-1]) / 2
        slope2 = (high[i] - low[i]) / 2
        if slope1 * slope2 < 0:
            if slope1 < 0:
                pattern[i] = -1
            else:
                pattern[i] = 1
    return pattern

sample_crypto_df['symmetrical_triangle'] = detect_symmetrical_triangle(sample_crypto_df['high'], sample_crypto_df['low'])
# Identify the occurrence of the pattern using the dataframe.loc method to find the index positions where the pattern occurs.
bullish_symmetrical_triangle_indexes = sample_crypto_df.loc[sample_crypto_df['symmetrical_triangle'] == 1].index
bearish_symmetrical_triangle_indexes = sample_crypto_df.loc[sample_crypto_df['symmetrical_triangle'] == -1].index
print(f'bullish_descending_triangle_indexes: {len(bullish_symmetrical_triangle_indexes)}, bearish_symmetrical_triangle_indexes: {len(bearish_symmetrical_triangle_indexes)}')


In [ ]:
sample_crypto_df

Finally, you can plot the descending triangle pattern on a candlestick chart using the mplfinance library. This will produce a candlestick chart with the SMA and RSI indicators plotted, along with the descending triangle pattern marked with green triangles for bullish pattern and red triangles for bearish pattern.

In [ ]:
sample_crypto_df_seg = sample_crypto_df.tail(100)
sample_crypto_df_seg 

In [ ]:
import mplfinance as mpf

df = sample_crypto_df_seg 

# Create a new DataFrame with just the open, high, low, and close columns
ohlc = df[['open', 'high', 'low', 'close']]

# Create a new DataFrame with just the pattern columns
patterns = df[['descending_triangle', 'double_bottom', 'double_top', 'head_and_shoulders', 'symmetrical_triangle']]

# Find the start and end indices of each block of candles with a pattern
pattern_blocks = []
current_block_start = None
for i in range(len(patterns)):
    if patterns.iloc[i].sum() > 0:
        if current_block_start is None:
            current_block_start = i
    else:
        if current_block_start is not None:
            pattern_blocks.append((current_block_start, i-1))
            current_block_start = None
if current_block_start is not None:
    pattern_blocks.append((current_block_start, len(patterns)-1))

# Create a list of data for the pattern blocks
data = []
for block_start, block_end in pattern_blocks:
    data.extend([(i, ohlc['low'].iloc[i], ohlc['high'].iloc[i]) for i in range(block_start, block_end+1)])

# Define the plot style for the pattern blocks
block_style = mpf.make_addplot(data, type='scatter', marker='s', markersize=150, 
                               markeredcolor='black', alpha=0.3, color='#b0c4de')

# Plot the candlestick chart with the pattern block style
mpf.plot(ohlc, type='candle', volume=True, figratio=(12, 6), figsize=(12, 9), 
         addplot=block_style)

